## 📊 分析の流れ
<details>
<summary><b>目次</b></summary>

1. **データ読み込み**
1. **欠損値処理**
    - 欠損列処理  
    - 欠損値処理  
    - 異常値処理  
1. **集計**
1. **データ出力**
</details>

In [ ]:
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import src.config as cfg
import src.data_loader as dl

## 📥 設定値の読み込み
yamlファイルから設定を読み込みます。また、プレースホルダやゼロをチェックします。


In [ ]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

## 🔍 欠損列の確認
4つファイルの列が各年度で同じかを目視で確認しておきます。
問題がある場合は処理します。
<details><summary><b>結果</b></summary>
すべてのファイルで同じ列が登録されています。問題ありません。

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>


In [ ]:
dl.load_yearly_header(settings["data_path"], settings["years"], settings["files"])

## 📥 欠損値表現のチェック
プレースホルダやゼロをチェックし、NaNのデータを特定します。  
NaNのデータは、setting.yamlの`na_values`で指定します。

```python


In [ ]:
# nan_valuesに""だけ指定し、他のプレースホルダをチェックする"
df = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"], na_values=[""])
#display(all_df)
counts = []
for filename in df:
    for col in df[filename]:
        #print(col)
        ser = df[filename][col].fillna('')
        counts.append(
            (filename, col,
            (ser == '-').sum(),
            (ser == '0').sum(),
            (ser == '—').sum(),
            ser.str.contains('[A-Za-z]', na=False).sum())
            )
placeholder_counts = pd.DataFrame(counts, columns=["filename","col","ハイフン半角","ゼロ","ハイフン全角","alphabet"])
display(placeholder_counts)


## 📥 データ読み込み

In [ ]:
all_df = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"], settings["na_values"])
#display(all_df[settings["files"][0]].head(1000))

## 🧹 欠損値の傾向を大まかに確認する
ここでは、後の分析・スクリーニングの処理を適切に設計するため、欠損値の傾向を確認します。  
<details><summary><b>分析内容</b></summary>

1. 列ごとの欠損値の割合を確認する
1. 年ごとの欠損率の傾向を確認する
1. 企業コードごとの欠損率の傾向を確認する
<br>
</details>
<details><summary><b>結果と考察</b></summary>

1. 列ごとの欠損値の割合
    * 短期借入金、長期借入金、自社株買いの欠損割合は高いため、その傾向を詳細分析
    * 重要指標であるROA、ROE、配当性向、純資産配当率の欠損傾向の詳細分析
    * ROA、ROE、配当性向、純資産配当率の置き換え・補足可能かを検討
1. 年ごとの欠損率の傾向
    * 設備投資の欠損率が2025年に上がっている原因の分析
1. 企業コードごとの欠損率の傾向を確認する
    * 半数以上を占める0.01-0.11の欠損パターンを確認

</details>


### ☑️ 列ごとの欠損値の割合を確認する

In [ ]:
df_combined = pd.DataFrame()
for filename in settings["files"]:
    df = all_df[filename].copy()
    missing_column_df = df.drop(columns=["コード", "年度"]).isna().mean()
    df_combined = pd.concat([df_combined, missing_column_df], axis=0)
fig = px.bar(df_combined, x=df_combined.index, y=df_combined[0], labels={"x":"項目","y":"欠損率"}, title="平均欠損率")
fig.show()

### ☑️ 年ごとの欠損率の傾向を確認

In [ ]:
df_combined = pd.DataFrame()
for filename in settings["files"]:
    df = all_df[filename].copy()
    df["年度_年"] = df["年度"].astype(str).str[:4].astype(int)

    missing_ratio_by_year = (
        df.drop(columns=["コード", "年度"])
        .groupby("年度_年")
        .agg(lambda g: g.isnull().mean())
        .reset_index()
    )
    missing_ratio_by_year = missing_ratio_by_year.set_index("年度_年")
    df_combined = pd.concat([df_combined, missing_ratio_by_year], axis=1)
#display(df_combined)
fig = px.line(
    df_combined,
    title="年度別 欠損値割合の推移",
    markers=True,
)
fig.show()

### ☑️ 企業コードごとの欠損率の傾向を確認

In [ ]:
# コードごとに平均欠損率を計算する
merged = all_df[settings["files"][0]]
for filename in settings["files"][1:]:
    df = all_df[filename]
    merged = pd.merge(merged, df, on=["コード", "年度"], how="outer")
#display(merged)
# コードごとに欠損率を計算
missing_score = (
    merged
    .drop("年度",axis=1)
    .groupby("コード")
    .agg(lambda x: x.isnull().mean())  # 各列ごとの欠損率
    .mean(axis=1)                      # 列平均
    .reset_index(name="平均欠損率")
)
#display(missing_score)
fig = px.histogram(missing_score, x="平均欠損率",nbins=50,
    title="全企業の欠損率分布", labels={"平均欠損率": "平均欠損率"},histnorm="percent")
fig.show()

## 🧽 欠損値の傾向を詳細に分析する
ここでは、大まかな傾向から、詳細に欠損値の傾向を確認します。  
<details><summary><b>分析内容</b></summary>

1. 短期借入金、長期借入金、自社株買いの欠損割合は高いため、その傾向を詳細分析
1. 重要指標であるROA、ROE、配当性向、純資産配当率の欠損傾向の詳細分析
1. ROA、ROE、配当性向、純資産配当率の置き換え・補足可能かを検討
1. 設備投資の欠損率が2025年に上がっている原因の分析
1. 半数以上を占める0.01-0.11の欠損パターンを確認

<br>
</details>
<details><summary><b>結果と考察</b></summary>

- fy-balance-sheet.csv
- fy-cash-flow-statement.csv
- fy-profit-and-loss.csv
- fy-stock-dividend.csv
</details>

### 短期借入金、長期借入金、自社株買いの欠損割合は高いため、その傾向を詳細分析

In [ ]:
# これらの列を含むデータフレームを作成
merged = all_df[settings["files"][0]]
for filename in settings["files"][1:]:
    df = all_df[filename]
    merged = pd.merge(merged, df, on=["コード", "年度"], how="outer")
#display(merged)
df_target = merged[["コード", "年度", "短期借入金", "長期借入金", "自社株買い"]].copy()
#display(df_target)

# 欠損率割合いの分布
missing_ratio_by_code = (
    df_target.groupby("コード")[["短期借入金", "長期借入金", "自社株買い"]]
      .apply(lambda g: g.isnull().mean())
      .reset_index()
)
#display(missing_ratio_by_code.info())
fig = px.histogram(missing_ratio_by_code, x=["短期借入金", "長期借入金", "自社株買い"],
    title="短期借入金、長期借入金、自社株買いの欠損率分布", labels={"value": "欠損率"}, barmode="group", histnorm="percent")
#fig.show()

# 各企業の欠損率を計算し、"常時欠損" を抽出
codes_list = {}
missing_ratio_by_code = missing_ratio_by_code.set_index("コード")
#display(missing_ratio_by_code)
for col in ["短期借入金", "長期借入金", "自社株買い"]:
    df = missing_ratio_by_code[col]
    df = df[(df == 1.0)]
    codes_list[col] = df.index.tolist()
    #display(len(codes_list[col]))
#display(codes_list)

# 業種に偏りがあるか確認
p = os.path.join(settings["data_path"], settings["files_reference"][0])
df = pd.read_csv(p,header=0,na_values=[])

# 全体データ
df_industry_33 = df[["コード","33業種区分"]].drop_duplicates().set_index("コード")
df_industry_33["区分"] = "全体"
df_industry_17 = df[["コード","17業種区分"]].drop_duplicates().set_index("コード")
df_industry_17["区分"] = "全体"
#display(df_industry_17)
#display(df_industry_33)

# subsetデータ
df_sub_33 = {}
for col in ["短期借入金", "長期借入金", "自社株買い"]:
    df_sub_33[col] = df[df["コード"].isin(codes_list[col])].copy()
    df_sub_33[col] = df_sub_33[col][["コード","33業種区分"]].drop_duplicates().set_index("コード")
    df_sub_33[col]["区分"] = f"常時欠損_{col}"
    #display(df_sub_33[col])
    
# 結合
df_combined = pd.concat([df_industry_33, df_sub_33["短期借入金"], df_sub_33["長期借入金"], df_sub_33["自社株買い"]])

# Plotly Expressで可視化
fig = px.histogram(
    df_combined,
    x="33業種区分",
    color="区分",
    barmode="group",
    histnorm="percent",  # ← 割合表示
    title="33業種区分別の構成比比較（全体 vs 抽出subset）"
)
fig.show()
# """
# 時価総額に偏りがあるか確認


"""target_file = files[0]

fig = px.histogram(
    missing_score[target_file],  # 最初のファイルのデータを使用
    x="平均欠損率",
    nbins=50,
    title="全企業の欠損率分布"+files[0],
    labels={"平均欠損率": "平均欠損率"}
)
#fig.show()
# 特定範囲の欠損率を持つコードを抽出
threshold_low = 0.215
threshold_high= 0.225
filtered_codes = missing_score[target_file][
    (missing_score[target_file]["平均欠損率"] >= threshold_low) &
    (missing_score[target_file]["平均欠損率"] < threshold_high)
]
#display(filtered_codes)
# 列別欠損率
df_by_code = all_df[target_file].set_index("コード") # 企業コードを index にする
codes = filtered_codes["コード"].tolist() # 企業コードをリスト化
subset = df_by_code.loc[df_by_code.index.isin(codes)] # 企業コードのデータ抽出
#print(df_by_code)
col_missing = subset.isnull().mean().sort_values(ascending=False) # 列別欠損率
#display(col_missing)

#年次別の欠損率推移確認
df = all_df[target_file]
year_all = (
    df.groupby('年度')[['短期借入金', '長期借入金']].apply(lambda d: d.isnull().mean())
    .apply(lambda d: d)
    .reset_index()
)
# 欠損率の推移を可視化
fig = px.line(
    year_all,
    x="年度",
    y=["短期借入金", "長期借入金"],  # 複数列を同時に表示
    markers=True,
    title="各列の欠損率の推移"
)
#fig.show()
# 各企業の年次ごとの欠損率を計算し、"常時欠損" と "変化あり" を分類
status = {}
code_groups = subset.groupby("コード")
for code, g in code_groups:
    rate_by_year = g[['短期借入金','長期借入金']].isnull().mean(axis=1)  # 年行ごとの欠損率
    if rate_by_year.nunique() == 1 and rate_by_year.iloc[0] == 1.0:
        status[code] = 'always_missing'
    else:
        status[code] = 'variable'
pd.Series(status).value_counts()
# variable の一部を表示して具体的な年次パターンを確認
#for code, st in list(status.items())[:100]:
    #if st == 'variable':
        #print(code)
        #display(code_groups.get_group(code)[['年度','短期借入金','長期借入金']].head(20))
# 業種に偏りがあるか確認
codes = filtered_codes["コード"].tolist()
p = os.path.join(base_path, files_reference[0])
df = pd.read_csv(p,header=0,na_values=[])
# 全体データ
df_all = df.copy()
df_all["区分"] = "全体"
# subsetデータ
df_sub = df[df["コード"].isin(codes)].copy()
df_sub["区分"] = "抽出subset"
# 結合
df_combined = pd.concat([df_all, df_sub])
# Plotly Expressで可視化
fig = px.histogram(
    df_combined,
    x="33業種区分",
    color="区分",
    barmode="group",
    histnorm="percent",  # ← 割合表示
    title="33業種区分別の構成比比較（全体 vs 抽出subset）"
)
#fig.show()

# # 時価総額に偏りがあるか確認
# 時価総額の推定モデル：推定時価総額 = (PER * EPS) * (自己資本 / BPS) * (ROE / 10 + 営業CFマージン)
# 4つのファイルを結合
df_combined = pd.DataFrame()
for filename in files:
    df=all_df[filename]
    df_combined = pd.concat([df_combined, df], axis=0)
#display(df_combined)
df = df_combined[["コード","年度","純利益","EPS","株主資本","BPS","ROE","営業CFマージン"]].copy()
def combine_nonnull(series):
    #そのグループ内で最初に非NaNの値を返す
    return series.dropna().iloc[0] if series.notna().any() else np.nan

# 「コード」「年度」でグループ化して統合
df_merged = (
    df.groupby(["コード", "年度"], as_index=False)
      .agg(combine_nonnull)
      .sort_values(["コード", "年度"])
      .reset_index(drop=True)
)
#display(df_merged.head(1000))


"""


In [91]:
p = os.path.join(settings["data_path"], settings["files_reference"][0])
df = pd.read_csv(p,header=0,na_values=[])
all = df["コード"].tolist()
set_all = set(all)
display(len(all))
data = all_df[settings["files"][0]]["コード"].unique().tolist()
set_data = set(data)
display(len(data))
common = set_all & set_data
only_a = list(set_all - set_data)
only_b = list(set_data - set_all)
display(len(common))
display(len(only_a))
display(len(only_b))


4411

4432

3819

592

613